---
Entrenamiento usando logistin regression - countvectorizer. Librerìa Sklearn con lo que aprendí en el curso de datacamp.


---



In [0]:
!ls

adc.json  data_test.txt  data_train.txt  sample  sample_data


In [0]:
!pip install Cython --install-option="--no-cython-compile"
!pip install fasttext 

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)


In [0]:
import fasttext
import numpy as np

**Si sample= True entonces guarda todo en un directorio que se llame sample** 

In [0]:
from pathlib import Path
sample = True
path = Path('./sample') if sample else Path('.')
path.mkdir(exist_ok=True)

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split 

Cargo los datos de google drive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1b59YbDir0lBELd1Y_8mzUpQQyYXPAzB0'})
download.GetContentFile(path/'train.csv')

In [0]:
!ls $path

train.csv




---


**Pre-processing data**:
Pre procesamiento de los datos -> verctorizer y wea / As simple as possible


---



In [0]:
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [0]:
def normalize_title(title):
    return unicodedata.normalize('NFKD', title.lower()).encode('ASCII', 'ignore').decode('utf8')
    

Carga Datos

In [0]:
df_train = pd.read_csv(path/'train.csv')


In [0]:
df_train['title']= df_train['title'].apply(normalize_title) # normalizo titulo
df_train = df_train[~df_train.title.isna() & (df_train.title != 'nan') & (df_train.title != '')]# me quedo solo con los que no son nulos

In [0]:
df_train['both'] ='__label__'+df_train.category + ' ' + df_train.title
df_train.head()

,title,label_quality,language,category,both
0,hidrolavadora lavor one 120 bar 1700w bomba a...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS,__label__ELECTRIC_PRESSURE_WASHERS hidrolavado...
1,placa de sonido - behringer umc22,unreliable,spanish,SOUND_CARDS,__label__SOUND_CARDS placa de sonido - behring...
2,maquina de lavar electrolux 12 kilos,unreliable,portuguese,WASHING_MACHINES,__label__WASHING_MACHINES maquina de lavar ele...
3,par disco de freio diant vent gol 8v 08/ frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS,__label__VEHICLE_BRAKE_DISCS par disco de frei...
4,flashes led pestanas luminoso falso pestanas p...,unreliable,spanish,FALSE_EYELASHES,__label__FALSE_EYELASHES flashes led pestanas ...


In [0]:
X_train, X_test = train_test_split(df_train['both'], test_size=0.4, random_state=53,stratify=df_train.category) # 1% por categoria
                                                 

In [0]:
X_test 

8298859     __label__ENERGETIC_STONES corrector de la cubi...
8460994          __label__SOLID_SWEET_PASTES rapadura mineira
17576672    __label__BRUSH_CUTTERS rocadeira profissional ...
14886754    __label__CRIBS berco americano bambini branco ...
2577443           __label__MICROCONTROLLERS plc siemens 24 rc
3273215     __label__ELECTRICAL_CABLES cabo flexivel antic...
8203510       __label__CUSHION_COVERS capa de almofada nordic
15296666    __label__FISHING_ROD_CASES cana de pesca volar...
17289585    __label__3D_PRINTER_FILAMENTS filamento flex i...
216224      __label__TOY_BUILDING_SETS blocky balde basico...
18290128    __label__CHARCOAL_GRILLS carrinho espetinho ch...
3755692     __label__AV_RECEIVERS amplificador residencial...
4397924          __label__LONGBOARDS longboard completo matza
16340526    __label__MATE_SETS especial tc7 chimarrao + te...
19029405    __label__MINI_COMPONENT_SYSTEMS microcomponent...
12654345    __label__COMPLETE_SKATEBOARDS skate santa cruz...
19849036

Saco columnas que no necesito

In [0]:
# only reliable
#df_train=df_train[df_train.label_quality=='reliable']
#df_train=df_train.drop(columns=['label_quality','language'])


In [0]:
import csv
X_train.to_csv(r'data_train.txt', header=None, index=None, sep='|', mode='a',quoting=csv.QUOTE_NONE,escapechar = ' ')
X_test.to_csv(r'data_test.txt', header=None, index=None, sep='|', mode='a',quoting=csv.QUOTE_NONE,escapechar = ' ')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
model = fasttext.train_supervised('data_train.txt')

In [0]:
model.predict('Philips Lumea Sc2003/11 - No Funciona',k=2)

(('__label__BOOKS', '__label__TRADING_CARD_GAMES'),
 array([0.3444671 , 0.17077014]))

---
**Entrenamiento**



---

In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
spanish_stopwords= stopwords.words('spanish')

In [0]:
count_vectorizer = CountVectorizer(stop_words=spanish_stopwords)
count_train = count_vectorizer.fit_transform(df_train['title'])


In [0]:
#tfidf_train = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
#tfidf_test = pd.DataFrame(tfidf_test.A, columns=tfidf_vectorizer.get_feature_names())

In [0]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [0]:
logit_classifier = LogisticRegression(solver="saga")

In [0]:
logit_classifier.fit(count_train,df_train['category'])

**Guardar modelo**

In [0]:
import pickle
pkl_filename = "logit_meli3.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(logit_classifier, file)

In [0]:
from google.colab import files
files.download('logit_meli3.pkl') 

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 37468, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

**Prediccion**

In [0]:
download = drive.CreateFile({'id': '1lRya91Zk9sDWanPynpvi0zTopvtGDJYZ'})
download.GetContentFile(path/'subm.csv')
df_subm = pd.read_csv(path/'subm.csv')
df_subm['title']= df_subm['title'].apply(normalize_title) # normalizo titulo
df_subm=df_subm.drop(columns=['language'])
#count_subm = count_vectorizer.transform(df_subm['title'])

In [0]:
!cat $path/subm.csv


Output hidden; open in https://colab.research.google.com to view.

In [0]:
pred = logit_classifier.predict(count_subm)

In [0]:
print(pred)

['DIAPER_BAGS' 'BABY_CHANGING_PADS' 'ENGINE_COOLING_FAN_MOTORS' ...
 'CALCULATORS' 'DINING_SETS' 'WASTE_BASKETS']


In [0]:
df_subm['category']=pred
df_subm.head()


,id,title,category
0,0,kit maternidade bolsa-mala baby/bebe vinho men...,DIAPER_BAGS
1,1,trocador de fraldas fisher price feminino rosa...,BABY_CHANGING_PADS
2,2,motor ventoinha - fiat idea / palio 1.8 - a 04...,ENGINE_COOLING_FAN_MOTORS
3,3,amortecedor mola batente d dir new civic 14 - ...,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,4,cadeirinha de carro bebe princesa princess 9 a...,BABY_CAR_SEATS


In [0]:
df_subm=df_subm.drop(columns=['title'])

In [0]:
df_subm.to_csv('submission8.csv',index=False)

**Download submission**

In [0]:
from google.colab import files
files.download('submission8.csv') 